# Internal Inconsistencies

In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from IPython.display import display
import import_ipynb
import data_preperation as dp
import copy
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import openpyxl
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


importing Jupyter notebook from data_preperation.ipynb


In [2]:
countries={ 'Austria': 'AT', 'Belgium': 'BE',  'Bulgaria': 'BG', 'Switzerland': 'CH', 'Czech Republic': 'CZ', 'Germany': 'DE', 'Denmark': 'DK', 'Estonia': 'EE', 'Spain': 'ES', 'Finland': 'FI', 'France': 'FR',  'Greece': 'GR', 'Hungary': 'HU', 'Ireland': 'IE', 'Italy': 'IT', 'Lithuania': 'LT', 'Latvia': 'LV', 'Montenegro': 'ME','Netherlands': 'NL', 'Norway': 'NO', 'Poland': 'PL', 'Portugal': 'PT', 'Serbia': 'RS', 'Sweden': 'SE', 'Slovenia': 'SI', 'Slovakia': 'SK', 'United Kingdom': 'UK'}

abbr_list=list(countries.values())

# load_data = dp.load(countries)
# generation_data = dp.generation(countries)
# cross_border_data = dp.cross_border(abbr_list)[1]
# import_export_using_crossborder_data = dp.import_export_using_crossborder(dp.cross_border(abbr_list)[0],abbr_list)

# 1. Internal inconsistencies based on unedited ENTSO-E data

## 1.1 Calculating missing values and mismatch analysis in data files

In [3]:
def omit_dst(df):

    # Due to day light saving, all the datasets have null values on 25th March from 02:00 - 03:00. 
    # Considering the time intervals each country update the data, a total number of rows of 4,2 or 1 are dropped from the 31st March, 02:00 - 03:00 time interval.
    # Also this returns a integer ('divider') based on the file length to get the energy values in a later step. In 15 min interval files this is 4, in 30 min interval files this is 2 and in 1 hour interval files this is 1.
    
    length = len(df)
    if length == 35044:
        df = df.drop(range(7976, 7980)).reset_index(drop=True)
        divider = 4
    elif length == 17522:
        df = df.drop(range(3988, 3990)).reset_index(drop=True)
        divider = 2
    else:
        df = df.drop(range(1994, 1995)).reset_index(drop=True)
        divider = 1
    
    return(df,divider)

def mismatch_raw_data(countries):

    load_missing_data = []
    generation_missing_data = []
    transmission_missing_data =[]
    transmission_data = pd.DataFrame()
    transmission_data_temp = pd.DataFrame()
    temp1 = pd.DataFrame()
    
    mismatch_data = {}
    temp = pd.DataFrame()
    csvs = glob.glob("../Data Sources/ENTSO-E/2018/Transmission/*.csv")

    # In the following command we read each csv file and drop the rows related to day light saving using 'omit_dst' function
    # We create a new dataframe 'transmission_data_temp' with all the columns in all the csvs attached together.
    # Then one by one we copy each csv to 'temp' dataframe and make all the 'n/e' values of the 'temp' 0.
    # Then we cretae two new column in the 'transmission_data' dataframe and name it as the two country codes the power transmission occurs.
    # We use string editing to get the two country codes from the file path.
    # Then we save the total number of null values in load data of each country in the load_missing_data list.
    # Then we save the total number of null values in generation data of each country in the generation_missing_data list.
    # Then we create a list of column names with all the columns of 'transmission_data_temp' if {country_abbreviations} is in the column name.
    # Then we save the total number of null values in each 'transmission_data_temp' dataframe of each country in the transmission_missing_data list using the filtered columns in the previous step.
    # Then we make a list of column heads of imports and exports associated with a given country_abbreviation
    # We calculate the annual data mismatch of each country as (generation + imports - load - exports) of the country.
    # Countries provide the data in 15 min, 30 min and 1 hour intervals.
    # Therefore to ge the energy values from the power values, we divide the total power data by 4,2 or 1 using the 'divider' variable as required.
    
    for csv in csvs:

        temp, divider_transmission = omit_dst(pd.read_csv(csv))
        transmission_data_temp = pd.concat([transmission_data_temp,temp.iloc[:,1:]],axis=1)

        temp=temp.replace(['n/e', np.nan], 0)
        transmission_data[f'{csv[42:44]} - > {csv[45:47]}'] = pd.to_numeric(temp.iloc[:, 2])/divider_transmission
        transmission_data[f'{csv[45:47]} - > {csv[42:44]}'] = pd.to_numeric(temp.iloc[:, 1])/divider_transmission
    
    for country,abbr in countries.items():
        load_data,divider_load = omit_dst(pd.read_csv(f'../Data Sources/ENTSO-E/2018/Load/{country}.csv'))
        generation_data, divider_gen = omit_dst(pd.read_csv(f'../Data Sources/ENTSO-E/2018/Generation/{country}.csv', low_memory=False))
        
        load_missing_data.append(load_data.isnull().sum().sum())
        generation_missing_data.append(generation_data.isnull().sum().sum())

        transmission_links = [x for x in transmission_data_temp.columns.values if abbr in x]
        transmission_missing_data.append(transmission_data_temp[transmission_links].isnull().sum().sum())

        imports=[x for x in transmission_data.columns.values if abbr in x[-2:]]
        exports=[x for x in transmission_data.columns.values if abbr in x[:2]]

        generation_data=generation_data.drop(['Hydro Pumped Storage  - Actual Consumption [MW]'],axis=1)

        mismatch_data.update({f'{abbr}': [generation_data.iloc[:, 2:].sum(axis=1).sum()/divider_gen, transmission_data[imports].sum(
            axis=1).sum(), load_data.iloc[:, 2].sum()/divider_load, transmission_data[exports].sum(axis=1).sum(), round(abs(generation_data.iloc[:, 2:].sum(axis=1).sum()/divider_gen + transmission_data[imports].sum(
                axis=1).sum() - load_data.iloc[:, 2].sum()/divider_load - transmission_data[exports].sum(axis=1).sum())/1000000, 2)]})

    temp1['Country'] = list(countries.values())
    temp1['No. of missing data in Load data'] = load_missing_data
    temp1['No. of missing data in Generation data'] = generation_missing_data
    temp1['No. of missing data in Transmission data'] = transmission_missing_data
  
    mismatch_data_copy = mismatch_data.copy()
    mismatch_data_copy = sorted(mismatch_data_copy.items(), key= lambda item: item[1][-1], reverse=True)
    display(temp1)   

    width = 0.35
    labels = [x[0] for x in mismatch_data_copy]
    X = np.arange(len(labels))
    plt.figure(figsize=(20,10))
    plt.bar([x for x in X], [x[1][-1] for x in mismatch_data_copy], width, color='aqua',edgecolor='black')
    plt.xlabel('Countries')
    plt.ylabel('Mismatch [TWh]')
    plt.title('Mismatch analysis based on (generation + imports - load - exports) in raw ENTSO-E data')
    plt.grid()
    plt.xticks(X,labels)
    plt.show()

    return mismatch_data


# 2. Consolidation based on internally gap filled ENTSO-E data

## 2.1 Polynomial linear regression based internal approach

In [11]:
# In the following section we take each generation,load and transmission csvs and send them to the 'function' function.
# This function send each dataframe to 'omit_dst' function and removes the empty observations on March 28th and get the 'divider' value which is associated with the length of the dataframe.
# Then we replace 'n/e' values with 0

def function(df):
    edited_df = pd.DataFrame()
    df, divider = omit_dst(df)
    df = df.replace('n/e', 0)

    # In the following section we take each generation,load and transmission csvs and send them to the 'neareset_neighbours_mean' function.
    # This function send each dataframe to 'omit_dst' function and removes the empty observations on March 28th and get the 'divider' value which is associated with the length of the dataframe.
    # Then we replace 'n/e' values with 0
    # Then we check the dataframe column by column.
    # First we filter the columns in which the whole column is not 0 in the dataframe.
    # Then we add the index and value of each value in the column to a list called 'column_data'
    # Then we check value by value in the column untill a null value is found.
    # If the value is a null value, we get all the values in that column which are the values associated with the same time step of the null value and the absolute difference of the indexes of that value and the null value to the 'temp_list'.
    # For example if the null value occurus at 10:00 - 10:15 time step of some day, all the values associated with 10:00 - 10:15 time step throughout the year and and the absolute difference of the indexes of those value and the null value go to the 'temp_list'
    # Then the 'temp_list' is sorted and we get all the indexes in the 'temp_list' except the index of the null value to a 2D numpy array 'X'.
    # We get all the values in the 'temp_list' except the value of the null value to a 1D numpy array 'y'
    # Then we divide 'X' and 'y' values in the ratio of 30% test and 70% train data.
    # We create an array of degree values from 1 to 10.
    # Then we iterate the 'degrees' one by one and create polynomial values of 'x_train' data called 'x_poly_train' based on the value of the degree
    # Then we fit the polynomial linear regression function using 'x_poly_train' data and 'y_train' data.
    # Then based on the polynomial function, using the 'x_poly_test' data we predict the values of the 'y_test' data
    # Then based on the predicted values and 'y_test" data we calculate the Root Mean Square Error.
    # Applying the last 4 steps for each degree value, we select the degree value which gives the Lowest Root Mean Square Error.
    # Then we fit the polynomial linear regression function again using that degree which gives the Lowest Root Mean Square Error.
    # Based on the polynimial function we get the predicted value of the null value.

    for column in df.columns.values:
        if not (df[column] == 0).all():
            column_data = [[index, value] for index, value in enumerate(df.loc[:, column])]
            for index, value in column_data:
                if pd.isnull(value):
                    temp_list = [item for item in column_data if (index-item[0]) % (24*divider) == 0]
                    temp_list.sort()
                    X = np.array([i[0] for i in temp_list[1:]]).reshape(len(temp_list)-1, 1)
                    y = [i[1] for i in temp_list[1:]]
                    y = [0 if pd.isna(x) else x for x in y]

                    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

                    degrees = np.arange(1, 11)
                    min_rmse, min_deg = 1e10, 0

                    for degree in degrees:

                        # Preparing polynomial Train features based on x_train
                        poly_features = PolynomialFeatures(degree=degree, include_bias=False)
                        x_poly_train = poly_features.fit_transform(x_train)

                        # Polynomial linear regression based on train data
                        poly_reg = LinearRegression()
                        poly_reg.fit(x_poly_train, y_train)

                        # Predicting y values and getting root mean squared error based on predicted y values and y_test values
                        x_poly_test = poly_features.fit_transform(x_test)
                        poly_predict = poly_reg.predict(x_poly_test)
                        poly_mse = mean_squared_error(y_test, poly_predict)
                        poly_rmse = np.sqrt(poly_mse)

                        # Selecting the best degree of the polynimial function based on lowest root mean squared error
                        if min_rmse > poly_rmse:
                            min_rmse = poly_rmse
                            min_deg = degree

                    # Fitting the regression function again based on the selected best degree above
                    poly_features = PolynomialFeatures(degree=min_deg, include_bias=False)
                    x_poly_train = poly_features.fit_transform(x_train)
                    poly_reg = LinearRegression()
                    poly_reg.fit(x_poly_train, y_train)

                    df.loc[index,column] = poly_reg.predict(poly_features.fit_transform([[0]]))[0]
            edited_df = pd.concat([edited_df, df.loc[:, column]], axis=1) 
    edited_df[edited_df<0]=0
    return edited_df

def regression(countries):
    for country, abbr in countries.items():
        df_load = pd.read_csv(f'../Data Sources/ENTSO-E/2018/Load/{country}.csv').iloc[:, 2:]
        df_load = df_load.rename(columns={df_load.columns[0]: 'demand'})
        load_consolidated = function(df_load)
        load_consolidated.to_csv(f'../Data Sources/output/Polynomial Linear Regression/Load/{abbr}.csv')

        df_gen = pd.read_csv(f'../Data Sources/ENTSO-E/2018/Generation/{country}.csv').iloc[:, 2:]
        df_gen = df_gen.drop(['Hydro Pumped Storage  - Actual Consumption [MW]'], axis=1)
        gen_consolidated = function(df_gen)
        gen_consolidated.to_csv(f'../Data Sources/output/Polynomial Linear Regression/Generation/{abbr}.csv')
    print('LOAD & GENERATION DATA CONSOLIDATED')

    transmission_consolidated = pd.DataFrame()
    csvs = glob.glob("../Data Sources/ENTSO-E/2018/Transmission/*.csv")
    for csv in csvs:
        df = pd.read_csv(csv).iloc[:, 1:]
        df = df.rename(columns={df.columns[0]: f'{csv[45:47]} - > {csv[42:44]}', df.columns[1]: f'{csv[42:44]} - > {csv[45:47]}'})
        transmission_consolidated = pd.concat([transmission_consolidated, function(df)], axis=1)
    transmission_consolidated.to_csv(f'../Data Sources/output/Polynomial Linear Regression/Transmission/all_transmissions.csv')
    print('TRANSMISSION DATA CONSOLIDATED')


In [12]:
# regression(countries)


LOAD & GENERATION DATA CONSOLIDATED
TRANSMISSION DATA CONSOLIDATED


## 2.2 Polynomial linear regression based optimization approach

In [5]:
# def alpha(load_dic, generation_dic, transmission_data, countries):

#     abbr_list = list(countries.values())
#     load_dic_copy = copy.deepcopy(load_dic)
#     generation_dic_copy = copy.deepcopy(generation_dic)
#     transmission_data_copy = copy.deepcopy(transmission_data)

#     for abbr in abbr_list:
#         load_dic[abbr] = load_dic[abbr].fillna(0.001)
#         generation_dic[abbr]=generation_dic[abbr].fillna(0.001)
    
#     transmission_data = transmission_data.fillna(0.001)

#     # In the following command, we calculate the alpha value by sending it to 'data_preperation.ipynb'

#     alpha = dp.calculate_alpha(load_dic_copy, generation_dic_copy, transmission_data_copy, abbr_list)
#     print('ALPHA CALCULATED')

#     # In the follwoing commands we create a 'country_index' with the names abbreviations of the 27 countries we consider.
#     # Then we create a 'time_index' which is a list of integers from 0 to 8760 which indicates the timesteps 
#     # Then we create a 'generation_index' which is a dictionary with country_abbreviations as keys and generation_sources os each country as the values of each key.
#     # Then we create a 'generation_fuels' which is a list of all the generation_sources we consider.
#     # Then we create 2 dictionaries 'imports_index' and 'exports_index' with country_abbreviations as the keys.
#     # In the column heads of 'transmission_data' dataframe, if the name of the key is in the first two characters of the column head, that column head goes as the value in 'export_index' dictionary
#     # In the column heads of 'transmission_data' dataframe, if the name of the key is in the last two characters of the column head, that column head goes as the value in 'import_index' dictionary
#     # Then we create a 'transmission_index' with all the column heads of 'transmission_data' dataframe.

#     country_index = list(countries.values())
#     time_index = np.arange(8760, dtype=int)

#     generation_index = {}

#     for abbr, df in generation_dic.items():
#         generation_index[abbr] = [x for x in df.columns.values]

#     generation_fuels = np.array(list(set([item for sublist in generation_index.values() for item in sublist])))
#     generation_fuels.sort()

#     imports_index = {}
#     exports_index = {}

#     for abbr in abbr_list:
#         imports_index[abbr] = [x for x in transmission_data.columns.values if abbr in x[-2:]]
#         exports_index[abbr] = [x for x in transmission_data.columns.values if abbr in x[:2]]

#     transmission_index = np.array(transmission_data.columns.values)
#     transmission_index.sort()

#     # In the following commands, we initiate the pyomo optimization with 'Gurobi' solver
#     # We declare the three variables 'delta_generation', 'delta_load' and 'delta_transmission'
#     # 'delta_generation' consists of 2,207,520 values which vary with country, generation_source and timestep.
#     # 'delta_load' consists of 236,520 values which vary with country and timestep.
#     # 'delta_transmission' consists of 858,480 values which vary with timestep and with country indirectly.
#     # We declare the model constraint as the sum of (delta_generation * generation + delta_transmission(imports) * imports) is equal to the sum of (delta_load * load + delta_transmission(exports) * exports) in all time steps.
#     # We declare objective function as to MINIMIZE the sum of (delta_generation^2 * alpha(generaion)) + sum of (delta_load^2 * alpha(load)) + sum of (delta_transmission^2 * alpha(transmission)) in all timesteps.
#     # Then solve the model.

#     model = pyo.ConcreteModel()

#     model.country_index = pyo.Set(initialize=country_index)
#     model.time_index = pyo.Set(initialize=time_index)
#     model.generation_fuels = pyo.Set(initialize=generation_fuels)
#     model.transmission_index = pyo.Set(initialize=transmission_index)

#     model.delta_generation = pyo.Var(model.country_index, model.time_index, model.generation_fuels, bounds=(0.0, None))
#     model.delta_load = pyo.Var(model.country_index, model.time_index, bounds=(0.0, None))
#     model.delta_transmission = pyo.Var(model.time_index, model.transmission_index, bounds=(0.0, None))

#     print('VARIABLES DECLARED')

#     def balance_rule(model, country, time):
#         return sum(model.delta_generation[country, time, generation] * generation_dic[country][generation][time] for generation in generation_index[country]) + sum(model.delta_transmission[time, link] * transmission_data[link][time] for link in imports_index[country]) ==  \
#             model.delta_load[country, time] * load_dic[country]["demand"][time] + \
#             sum(model.delta_transmission[time, link] * transmission_data[link][time] for link in exports_index[country])
#     model.balance_rule = pyo.Constraint(model.country_index, model.time_index, rule=balance_rule)

#     def ObjRule(model):
#         return sum(model.delta_generation[country, time, generation] ** 2 * float(alpha[country][generation].iloc[time]) for country in model.country_index for time in model.time_index for generation in generation_index[country]) \
#             + sum(model.delta_transmission[time, link] ** 2 * float(alpha["transmission_data"][link].iloc[time]) for time in model.time_index for link in model.transmission_index)\
#             + sum(model.delta_load[country, time] ** 2 * float(alpha[country]["demand"].iloc[time]) for country in model.country_index for time in model.time_index)

#     model.obj = pyo.Objective(rule=ObjRule, sense=pyo.minimize)
#     opt = SolverFactory("gurobi", solver_io="python")
#     opt.solve(model)
#     print('OPTIMIZATION COMPLETED')

#     # In the following commands we copy the pyomo results into different intermediary variables.
#     # We create 'intermediary_var' dictionary and inside it create two other dictionaries as keys called 'generation' and 'load' and create a dataframe as another key called 'transmission'.
#     # We create 'unit_var' dictionary and inside it create two other dictionaries as keys called 'generation' and 'load' and create a dataframe as another key called 'transmission'.
#     # We fill the 'intermediary_var["generation"]' from the 'delta_generation' values and 'intermediary_var["load"]' with 'delta_load' values and 'intermediary_var["transmission"]' from 'delta_transmission' values.
#     # We fill 'unit_var["generation"]', 'unit_var["load"]' and 'unit_var["transmission"]' which have the same size as of 'intermediary_var["generation"]', 'intermediary_var["load"]' and ''intermediary_var["transmission"]' with integer 1.

#     generation_index_copy = copy.deepcopy(generation_index)

#     intermediary_var = {}
#     intermediary_var["generation"] = {}
#     intermediary_var["load"] = {}
#     unit_var = {}
#     unit_var["generation"] = {}
#     unit_var["load"] = {}

#     for country in country_index:
#         table_gen = []
#         row_load = []
#         for time in time_index:
#             row_gen = []
#             for generation in generation_index_copy[country]:
#                 row_gen.append(model.delta_generation[country, time, generation].value)
#             table_gen.append(row_gen)
#             row_load.append(model.delta_load[country, time].value)

#         intermediary_var["generation"][country] = pd.DataFrame.from_records(table_gen)
#         intermediary_var["load"][country] = pd.DataFrame(row_load)
#         intermediary_var["generation"][country].columns = generation_index_copy[country]
#         intermediary_var["load"][country].columns = ['demand']
#         intermediary_var["generation"][country].to_csv("../Data Sources/output/Alpha/Debugging/" + "alpha_gen_" + country + ".csv")
#         intermediary_var["load"][country].to_csv("../Data Sources/output/Alpha/Debugging/" + "alpha_load_" + country + ".csv")
#         unit_var["generation"][country] = pd.DataFrame(1, index=np.arange(df.shape[0]), columns=generation_index_copy[country])
#         unit_var["load"][country] = pd.DataFrame(1, index=np.arange(df.shape[0]), columns=['demand'])

#     table_transmission = []

#     for time in time_index:
#         row = []
#         for link in transmission_index:
#             row.append(model.delta_transmission[time, link].value)
#         table_transmission.append(row)
#     intermediary_var["transmission"] = pd.DataFrame.from_records(table_transmission)
#     intermediary_var["transmission"].columns = transmission_index
#     intermediary_var["transmission"].to_csv("../Data Sources/output/Alpha/Debugging/alpha_transmission.csv")
#     unit_var["transmission"] = pd.DataFrame(1, index=np.arange(transmission_data.shape[0]), columns=transmission_data.columns)

#     generation_dic_copy = copy.deepcopy(generation_dic)
#     load_dic_copy = copy.deepcopy(load_dic)
#     transmission_data_copy = copy.deepcopy(transmission_data)

#     # We send the intermediary_var and unit_var to 'data_preperation.ipynb' to get the consolidated generation,load and transmission values.

#     dp.data_consolidation('Alpha',generation_dic_copy, load_dic_copy, transmission_data_copy, intermediary_var, unit_var)


## 2.3 Moving average based optimization approach

In [7]:
def sigma(load_dic, generation_dic, transmission_data, countries):

    abbr_list = list(countries.values())
    load_dic_copy = copy.deepcopy(load_dic)
    generation_dic_copy = copy.deepcopy(generation_dic)
    transmission_data_copy = copy.deepcopy(transmission_data)

    # In the following command, we calculate the sigma value by sending it to 'data_preperation.ipynb'

    sigma = dp.calculate_sigma(load_dic_copy, generation_dic_copy, transmission_data_copy, abbr_list)
    print('SIGMA CALCULATED')

    # In the follwoing commands we create a 'country_index' with the names abbreviations of the 27 countries we consider.
    # Then we create a 'time_index' which is a list of integers from 0 to 8760 which indicates the timesteps 
    # Then we create a 'generation_index' which is a dictionary with country_abbreviations as keys and generation_sources os each country as the values of each key.
    # Then we create a 'generation_fuels' which is a list of all the generation_sources we consider.
    # Then we create 2 dictionaries 'imports_index' and 'exports_index' with country_abbreviations as the keys.
    # In the column heads of 'transmission_data' dataframe, if the name of the key is in the first two characters of the column head, that column head goes as the value in 'export_index' dictionary
    # In the column heads of 'transmission_data' dataframe, if the name of the key is in the last two characters of the column head, that column head goes as the value in 'import_index' dictionary
    # Then we create a 'transmission_index' with all the column heads of 'transmission_data' dataframe.

    country_index = list(countries.values())
    time_index = np.arange(8760, dtype=int)

    generation_index = {}

    for abbr, df in generation_dic.items():
        generation_index[abbr] = [x for x in df.columns.values]

    generation_fuels = np.array(list(set([item for sublist in generation_index.values() for item in sublist])))
    generation_fuels.sort()

    imports_index = {}
    exports_index = {}

    for abbr in abbr_list:
        imports_index[abbr] = [x for x in transmission_data.columns.values if abbr in x[-2:]]
        exports_index[abbr] = [x for x in transmission_data.columns.values if abbr in x[:2]]

    transmission_index = np.array(transmission_data.columns.values)
    transmission_index.sort()

    # In the following commands, we initiate the pyomo optimization with 'Gurobi' solver
    # We declare the three variables 'delta_generation', 'delta_load' and 'delta_transmission'
    # 'delta_generation' consists of 2,207,520 values which vary with country, generation_source and timestep.
    # 'delta_load' consists of 236,520 values which vary with country and timestep.
    # 'delta_transmission' consists of 858,480 values which vary with timestep and with country indirectly.
    # We declare the model constraint as the sum of (delta_generation + generation + delta_transmission(imports) + imports) is equal to the sum of (delta_load + load + delta_transmission(exports) + exports) in all time steps.
    # We declare objective function as to MINIMIZE the sum of (delta_generation^2 * sigma(generaion)) + sum of (delta_load^2 * sigma(load)) + sum of (delta_transmission^2 * sigma(transmission)) in all timesteps.
    # Then solve the model.

    model = pyo.ConcreteModel()

    model.country_index = pyo.Set(initialize=country_index)
    model.time_index = pyo.Set(initialize=time_index)
    model.generation_fuels = pyo.Set(initialize=generation_fuels)
    model.transmission_index = pyo.Set(initialize=transmission_index)

    model.delta_generation = pyo.Var(model.country_index, model.time_index, model.generation_fuels, bounds=(0.0, None))
    model.delta_load = pyo.Var(model.country_index, model.time_index, bounds=(0.0, None))
    model.delta_transmission = pyo.Var(model.time_index, model.transmission_index, bounds=(0.0, None))

    print('VARIABLES DECLARED')

    def balance_rule(model, country, time):
        return sum(model.delta_generation[country, time, generation] + generation_dic[country][generation][time] for generation in generation_index[country]) + sum(model.delta_transmission[time, link] + transmission_data[link][time] for link in imports_index[country]) ==  \
            model.delta_load[country, time] + load_dic[country]["demand"][time] + \
            sum(model.delta_transmission[time, link] + transmission_data[link][time] for link in exports_index[country])
    model.balance_rule = pyo.Constraint(model.country_index, model.time_index, rule=balance_rule)

    def ObjRule(model):
        return sum(model.delta_generation[country, time, generation] ** 2 * float(sigma[country][generation].iloc[time]) for country in model.country_index for time in model.time_index for generation in generation_index[country]) \
            + sum(model.delta_transmission[time, link] ** 2 * float(sigma["transmission_data"][link].iloc[time]) for time in model.time_index for link in model.transmission_index)\
            + sum(model.delta_load[country, time] ** 2 * float(sigma[country]["demand"].iloc[time]) for country in model.country_index for time in model.time_index)

    model.obj = pyo.Objective(rule=ObjRule, sense=pyo.minimize)
    opt = SolverFactory("gurobi", solver_io="python")
    opt.solve(model)
    print('OPTIMIZATION COMPLETED')

    # In the following commands we copy the pyomo results into different intermediary variables.
    # We create 'intermediary_var' dictionary and inside it create two other dictionaries as keys called 'generation' and 'load' and create a dataframe as another key called 'transmission'.
    # We create 'unit_var' dictionary and inside it create two other dictionaries as keys called 'generation' and 'load' and create a dataframe as another key called 'transmission'.
    # We fill the 'intermediary_var["generation"]' from the 'delta_generation' values and 'intermediary_var["load"]' with 'delta_load' values and 'intermediary_var["transmission"]' from 'delta_transmission' values.
    # We fill 'unit_var["generation"]', 'unit_var["load"]' and 'unit_var["transmission"]' which have the same size as of 'intermediary_var["generation"]', 'intermediary_var["load"]' and ''intermediary_var["transmission"]' with integer 1.

    generation_index_copy = copy.deepcopy(generation_index)

    intermediary_var = {}
    intermediary_var["generation"] = {}
    intermediary_var["load"] = {}
    unit_var = {}
    unit_var["generation"] = {}
    unit_var["load"] = {}

    for country in country_index:
        table_gen = []
        row_load = []
        for time in time_index:
            row_gen = []
            for generation in generation_index_copy[country]:
                row_gen.append(model.delta_generation[country, time, generation].value)
            table_gen.append(row_gen)
            row_load.append(model.delta_load[country, time].value)

        intermediary_var["generation"][country] = pd.DataFrame.from_records(table_gen)
        intermediary_var["load"][country] = pd.DataFrame(row_load)
        intermediary_var["generation"][country].columns = generation_index_copy[country]
        intermediary_var["load"][country].columns = ['demand']
        intermediary_var["generation"][country].to_csv("../Data Sources/output/Sigma/Debugging/" + "sigma_gen_" + country + ".csv")
        intermediary_var["load"][country].to_csv("../Data Sources/output/Sigma/Debugging/" + "sigma_load_" + country + ".csv")
        unit_var["generation"][country] = pd.DataFrame(1, index=np.arange(df.shape[0]), columns=generation_index_copy[country])
        unit_var["load"][country] = pd.DataFrame(1, index=np.arange(df.shape[0]), columns=['demand'])

    table_transmission = []

    for time in time_index:
        row = []
        for link in transmission_index:
            row.append(model.delta_transmission[time, link].value)
        table_transmission.append(row)
    intermediary_var["transmission"] = pd.DataFrame.from_records(table_transmission)
    intermediary_var["transmission"].columns = transmission_index
    intermediary_var["transmission"].to_csv("../Data Sources/output/Sigma/Debugging/sigma_transmission.csv")
    unit_var["transmission"] = pd.DataFrame(1, index=np.arange(transmission_data.shape[0]), columns=transmission_data.columns)

    generation_dic_copy = copy.deepcopy(generation_dic)
    load_dic_copy = copy.deepcopy(load_dic)
    transmission_data_copy = copy.deepcopy(transmission_data)

    # We send the intermediary_var and unit_var to 'data_preperation.ipynb' to get the consolidated generation,load and transmission values.

    dp.data_consolidation('Sigma',generation_dic_copy, load_dic_copy, transmission_data_copy, intermediary_var, unit_var)

## 2.4 Mismatch analysis in the consolidated ENTSO-E data

In [8]:
# Similar to 'omit_dst' function above, we use 'hourly_data' function to get the divider value based on dataframe length

def hourly_data(df):
    length = len(df.index)
    if length == 35040:
        divider = 4
    elif length == 17520:
        divider = 2
    else:
        divider = 1
    return (df, divider)

# In the following function we get the ENTSO-E statistical factsheet of 2018 to a dataframe called 'stat_data'
# Then we selct only the country_name, generation,load,transmission_imports and transmission_exports columns and convert import,exports values to TWh which are stated in GWh
# We create a list with the names of the three folders where the consolidated csvs are saved in each internal consolidation method.
# We create another list with the names of the two folders where the consolidated csvs of 'Generation' and 'Load' files are saved.
# Then we iterate through the rows of 'stat_data' dataframe.
# We check if the country_abbreviation in 'stat_data''s country column is inside the 'selected_countries' list.
# Then we get generation and load data of the raw unedited data from 'mismatch_data' dictionary and generation and load data of Nearest Neighbour Mean, Polynomial Linear Regression and Optimization based methods from their csv files in the respective folders.
# We get the total sum value in each csv and get the annual energy value using the 'divider' variable and save the value in 'stat_data' dataframe.
# Similarly we get the transmission imports and exports from the respective csvs by filtering the country_abbreviations in the column headers.

def mismatch_analysis(mismatch_data,selected_countries):
    stat_data = pd.read_excel('../Data Sources/ENTSO-E/entsoe_sfs2018_web.xlsx').iloc[1:28, ].reset_index(drop=True)
    stat_data = stat_data[['Country', 'Total net generation','Consumption', 'Sum of imports', 'Sum of exports']]
    stat_data.iloc[:, 3:5] = stat_data.iloc[:, 3:5]/1000

    methods = ['Nearest Neighbours Mean','Alpha', 'Sigma']
    types = ['Generation', 'Load']

    for index, value in enumerate(stat_data.loc[:, 'Country']):

        if value in selected_countries:

            stat_data.loc[index, 'Raw_gen'] = mismatch_data[value][0]/1000000
            stat_data.loc[index, 'Raw_load'] = mismatch_data[value][2]/1000000
            stat_data.loc[index, 'Raw_imports'] = mismatch_data[value][1]/1000000
            stat_data.loc[index, 'Raw_exports'] = mismatch_data[value][3]/1000000

            for method in methods:
                for type in types:

                    df, divider = hourly_data(pd.read_csv(f'../Data Sources/output/{method}/{type}/{value}.csv'))
                    stat_data.loc[index, f'{type}_{method}'] = df.iloc[:, 1:].sum(axis=1).sum()/(divider*1000000)

                df = pd.read_csv(f'../Data Sources/output/{method}/Transmission/all_transmissions.csv').iloc[:, 1:]

                imports = [x for x in df.columns.values if value in x[-2:]]
                exports = [x for x in df.columns.values if value in x[:2]]

                stat_data.loc[index, f'Imports_{method}'] = df[imports].sum(axis=1).sum()/1000000
                stat_data.loc[index, f'Exports_{method}'] = df[exports].sum(axis=1).sum()/1000000
        else:
            stat_data = stat_data.drop(labels=index, axis=0)
    stat_data = stat_data.reset_index(drop=True)

    # We create new variable list xi where i belongs to [1,2,3,4]
    # Each xi list contains 4 values
    # After the iteration is completed x1 has the sum values of generation,load,imports and exports of raw unedited data of all the countries with the percentage difference of each of the 4 values compared to the respective generation,load,imports and exports sum values of ENTSO-E statistical data.
    # Similarly x2 has the percentage dofference of the 4 values of generation,load,imports and exports of Nearest Neighbour Mean method.
    # In the same way x3 is the Polynmial Linear regression method and x4 is the optimization  based method. 

    for i in range(1, 5):
        globals()[f'x{i}'] = []
        for j in range(1, 5):
            globals()[f'x{i}'].append(abs(stat_data.iloc[:, j].sum()-stat_data.iloc[:, i*4+j].sum())*100/stat_data.iloc[:, j].sum())

    width = 0.18
    labels = ['Actual generation per type', 'Actual load','Physical flow imports', 'Physical flow exports']
    X = np.arange(4)
    plt.figure(figsize=(10, 5))
    plt.bar([x-3*width/2 for x in X], x1, width, color='magenta', edgecolor='black', label='Raw data')
    plt.bar([x-width/2 for x in X], x2, width, color='aqua', edgecolor='black', label='Nearest Neighbours Mean consolidated data')
    plt.bar([x+width/2 for x in X], x3, width, color='gold', edgecolor='black', label='Polynomial Linear Regression consolidated data')
    plt.bar([x+3*width/2 for x in X], x4, width, color='lightcoral', edgecolor='black', label='Internal Sigma consolidated data')
    plt.xlabel('Data Items')
    plt.ylabel('Percentage difference')
    plt.title('Percentage difference with respect to ENTSO-E statistical data - 2018')
    plt.legend()
    plt.grid()
    plt.xticks(X, labels)
    plt.show()
